In [1]:
import urllib2, sys
from bs4 import BeautifulSoup
import pandas as pd
from io import StringIO
from IPython.core.display import HTML
import requests
import time

In [50]:
def get_teamID(team_name):
    team_info = pd.read_csv('../../scrapped_data/team_info.csv', index_col=0)
    teamID = team_info.loc[team_info.team_name == team_name, 'team_id'].tolist()[0]
    return teamID

In [32]:
def get_soup(url):
    hdr = {'User-Agent': 'Mozilla/5.0'}
    req = urllib2.Request(url,headers=hdr)
    page = urllib2.urlopen(req)
    soup = BeautifulSoup(page)
    return soup

In [42]:
def get_match_urls(params):
    done=False
    params['offset'] = 0
    urls = []
    while not done:
        match_page = "https://www.hltv.org/results?offset={offset}&content=demo&team={teamID}&startDate={startDate}&endDate={endDate}".format(**params)
        soup = get_soup(match_page)
        matches = soup.find_all("div", class_='results-all')
        
        if len(matches) == 0:
            break
        
        results = matches[0].find_all("a", class_="a-reset")
        urls  += [result['href'] for result in results]
        if len(urls) % 100 != 0:
            done = True
        else:
            params['offset'] += 100
    del params['offset']
    return urls

In [52]:
def parse_match(site):
    hdr = {'User-Agent': 'Mozilla/5.0'}
    site = 'https://www.hltv.org' + site
    req = urllib2.Request(site,headers=hdr)
    page = urllib2.urlopen(req)
    soup = BeautifulSoup(page)
    
    req = urllib2.Request(site,headers=hdr)
    page = urllib2.urlopen(req)
    soup = BeautifulSoup(page)

    demo_url = 'https://www.hltv.org' + soup.find_all("a", class_="flexbox left-right-padding")[0]['href']
    
    try:
        vetos = soup.find_all("div", class_="standard-box veto-box")[1].find_all("div")[0].find_all("div")
        vetos = [veto.text for veto in vetos]
    except:
        vetos = None
        
    stats_url = 'https://www.hltv.org' + [a_element['href'] for a_element in soup.find_all("a") if a_element.text == "Detailed stats"][0]

    tables = soup.find_all("div", class_ = "stats-content", id="all-content")[0]
    tables = tables.find_all("table")
    r = requests.get(site, headers=hdr)
    tables = pd.read_html(r.text, header=0)
    team_a, team_b = tables[0], tables[1]

    return {
        'url': site,
        'vetos': vetos,
        'stats_url': stats_url,
        'teams': [team_a, team_b]
    }

In [53]:
def scrape_match_data(team_name, startDate, endDate):
    teamID = get_teamID(team_name)
    params = {
        'teamID':teamID,
        'startDate':startDate,
        'endDate':endDate
    }
    urls = get_match_urls(params)
    matches = []
    for idx, url in enumerate(urls):
        matches.append(parse_match(url))
        time.sleep(5)
        print 'match {0} done'.format(idx)
    return matches

In [44]:
#testing function

team_name = 'TyLoo'
startDate ='2017-08-01'
endDate ='2017-08-01'

189